In [1]:
import numpy as np
import scipy.spatial as spatial
import shapes
import plotly.graph_objects as go

In [13]:
s = shapes.SuperQuadric("superellipsoid", np.random.uniform(0.01, 4, 3), [1, 1, 1, 3])
points = s.compute_points()
faces = s.compute_faces()
fig = go.Figure(data=[go.Mesh3d(x=points[0], y=points[1], z=points[2], color='lightpink', opacity=0.50)])
fig.show()